In [1]:
import os
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# Absolute path to the raw AEMET data directory
RAW_AEMET_DIR = os.path.abspath("../data/raw/aemet")

dfs = []
for fname in os.listdir(RAW_AEMET_DIR):
    if fname.endswith(".csv"):
        df = pd.read_csv(os.path.join(RAW_AEMET_DIR, fname))
        dfs.append(df)

meteo = pd.concat(dfs, ignore_index=True)


In [3]:
# Parsing dates
meteo["fecha"] = pd.to_datetime(meteo["fecha"], errors="coerce")
meteo = meteo.dropna(subset=["fecha"])
meteo["year"] = meteo["fecha"].dt.year

# Convert numerical columns
num_cols = ["tmed","tmax","tmin","prec","sol","presMax","presMin","hrMedia","altitud"]
for col in num_cols:
    if col in meteo.columns:
        meteo[col] = pd.to_numeric(meteo[col], errors="coerce")


In [4]:
# Filter years with at least 300 days of data
valid_years = (
    meteo.groupby(["nombre","year"])["tmin"]
    .count()
    .reset_index(name="n_days")
)
valid_years = valid_years[valid_years["n_days"] >= 300]

meteo_clean = meteo.merge(valid_years[["nombre","year"]], on=["nombre","year"])


In [5]:
# Compute annual variables
annual_vars = (
    meteo_clean
    .groupby(["nombre","year"], as_index=False)
    .agg({
        "tmin":"mean",
        "tmax":"mean",
        "tmed":"mean",
        "hrMedia":"mean",
        "sol":"mean",
        "presMax":"mean",
        "presMin":"mean",
    })
)

annual_vars["ATD"] = annual_vars["tmax"] - annual_vars["tmin"]


In [6]:
# Compute UHI
def compute_uhi(annual_df, urban_station, rural_station="MONTSERRAT"):
    urb = annual_df[annual_df["nombre"] == urban_station][["year","tmin"]].rename(columns={"tmin":"tmin_urb"})
    rur = annual_df[annual_df["nombre"] == rural_station][["year","tmin"]].rename(columns={"tmin":"tmin_rur"})
    merged = pd.merge(urb, rur, on="year", how="inner")
    merged["UHI"] = merged["tmin_urb"] - merged["tmin_rur"]
    merged["station"] = urban_station
    return merged

stations = {
    "Drassanes (urban core)": "BARCELONA, DRASSANES",
    "Fabra (urban-high)": "BARCELONA, FABRA",
    "BCN Airport": "BARCELONA AEROPUERTO",
    "Sabadell Airport": "SABADELL AEROPUERTO",
}

uhi_series = {
    label: compute_uhi(annual_vars, code)
    for label, code in stations.items()
}


In [7]:
# Merge UHI with annual variables
def merge_with_annual_vars(uhi_df, annual_df, station_code):
    urb_vars = annual_df[annual_df["nombre"] == station_code]
    return pd.merge(
        uhi_df,
        urb_vars[["year","tmin","tmax","tmed","ATD","hrMedia","sol","presMax","presMin"]],
        on="year",
        how="left"
    )

merged = {
    label: merge_with_annual_vars(uhi_df, annual_vars, stations[label])
    for label, uhi_df in uhi_series.items()
}


In [8]:
# Correlation analysis
def safe_corr(df, x_col, y_col="UHI"):
    sub = df[[x_col, y_col]].dropna()
    n = len(sub)
    if n < 2:
        return np.nan, np.nan, n
    r, p = pearsonr(sub[x_col], sub[y_col])
    return r, p, n

variables = [
    ("tmin","Tmin"),
    ("tmax","Tmax"),
    ("tmed","Tmed"),
    ("ATD","ATD"),
    ("hrMedia","RH"),
    ("sol","Sol"),
    ("presMax","presMax"),
    ("presMin","presMin"),
]

rows = []
for label, df in merged.items():
    row = {"Station": label}
    for col, col_label in variables:
        if col in df.columns:
            r, p, n = safe_corr(df, col)
            row[f"r(UHI,{col_label})"] = round(r,3) if not np.isnan(r) else np.nan
            row[f"p(UHI,{col_label})"] = round(p,4) if not np.isnan(p) else np.nan
    row["N years"] = len(df["year"].unique())
    rows.append(row)

corr_summary = pd.DataFrame(rows)
corr_summary


,Station,"r(UHI,Tmin)","p(UHI,Tmin)","r(UHI,Tmax)","p(UHI,Tmax)","r(UHI,Tmed)","p(UHI,Tmed)","r(UHI,ATD)","p(UHI,ATD)","r(UHI,RH)","p(UHI,RH)","r(UHI,Sol)","p(UHI,Sol)","r(UHI,presMax)","p(UHI,presMax)","r(UHI,presMin)","p(UHI,presMin)",N years
0,Drassanes (urban core),0.195,0.5231,-0.230,0.4494,0.180,0.5566,-0.612,0.0262,0.055,0.8579,NaN,NaN,NaN,NaN,NaN,NaN,13
1,Fabra (urban-high),0.377,0.0107,0.096,0.5285,0.233,0.1230,-0.411,0.0050,0.099,0.5344,0.253,0.0932,-0.481,0.0013,-0.432,0.0043,45
2,BCN Airport,0.753,0.0000,0.558,0.0001,0.699,0.0000,-0.711,0.0000,-0.585,0.0000,-0.228,0.1317,-0.112,0.4622,-0.224,0.1386,45
3,Sabadell Airport,0.812,0.0000,0.143,0.4753,0.629,0.0004,-0.834,0.0000,-0.180,0.5957,NaN,NaN,-0.659,0.5422,-0.772,0.4388,27


In [9]:
# Save correlation summary
out_dir = os.path.abspath("../reports/synthesis")
os.makedirs(out_dir, exist_ok=True)
corr_summary.to_csv(os.path.join(out_dir, "corr_summary_uhi_all_vars.csv"), index=False)

# Display formatted correlation summary
corr_summary.style.format(precision=3)



,Station,"r(UHI,Tmin)","p(UHI,Tmin)","r(UHI,Tmax)","p(UHI,Tmax)","r(UHI,Tmed)","p(UHI,Tmed)","r(UHI,ATD)","p(UHI,ATD)","r(UHI,RH)","p(UHI,RH)","r(UHI,Sol)","p(UHI,Sol)","r(UHI,presMax)","p(UHI,presMax)","r(UHI,presMin)","p(UHI,presMin)",N years
0,Drassanes (urban core),0.195,0.523,-0.230,0.449,0.180,0.557,-0.612,0.026,0.055,0.858,nan,nan,nan,nan,nan,nan,13
1,Fabra (urban-high),0.377,0.011,0.096,0.528,0.233,0.123,-0.411,0.005,0.099,0.534,0.253,0.093,-0.481,0.001,-0.432,0.004,45
2,BCN Airport,0.753,0.000,0.558,0.000,0.699,0.000,-0.711,0.000,-0.585,0.000,-0.228,0.132,-0.112,0.462,-0.224,0.139,45
3,Sabadell Airport,0.812,0.000,0.143,0.475,0.629,0.000,-0.834,0.000,-0.180,0.596,nan,nan,-0.659,0.542,-0.772,0.439,27


🔎 Síntesis del papel de cada variable en la UHI de Barcelona

A partir de la tabla de correlaciones podemos extraer una visión integrada de qué variables explican mejor la intensidad de la Isla de Calor Urbana (UHI) en el área de Barcelona:

Temperatura mínima (Tmin)

En las estaciones periurbanas y de valle (BCN Airport, Sabadell) la correlación entre UHI y Tmin es alta y positiva: los años con mínimas más elevadas son también los años con UHI más intensa.

Esto confirma que la UHI es, sobre todo, un fenómeno nocturno, ligado a la falta de enfriamiento durante la noche.

En el centro urbano (Drassanes) la relación es mucho más débil, lo que indica que allí la UHI está más condicionada por la geometría urbana, los materiales y el calor antropogénico que por la variabilidad interanual de Tmin.

Amplitud térmica diaria (ATD = Tmax − Tmin)

La ATD presenta una correlación negativa y significativa con la UHI en prácticamente todas las estaciones: cuando la amplitud diaria disminuye (las noches enfrían menos), la UHI aumenta.

Este resultado es muy consistente con la teoría de UHI: la clave no es tanto que el día sea más cálido, sino que la ciudad retiene el calor y reduce el enfriamiento nocturno.

Temperatura máxima (Tmax) y media (Tmed)

Tmax suele mostrar correlaciones más débiles con UHI: el calentamiento diurno afecta tanto a zonas urbanas como rurales y no discrimina tan bien la señal urbana.

Tmed presenta correlaciones intermedias, principalmente porque combina la información de Tmin y Tmax; aun así, es menos informativa físicamente que Tmin o ATD.

Humedad relativa (hrMedia)

En conjunto, la humedad muestra correlaciones débiles o no significativas con UHI, especialmente en el núcleo urbano.

Esto indica que, en Barcelona, la UHI está dominada por los factores térmicos y estructurales más que por la disponibilidad de humedad.

En algunas estaciones abiertas (p. ej. aeropuerto) la relación puede ser algo más clara, pero sigue siendo secundaria frente a Tmin y ATD.

Insolación (sol)

Solo está disponible en Fabra, BCN Airport y Montserrat.

En Fabra se observa una relación inversa moderada entre sol y UHI, mientras que en el aeropuerto la correlación es débil.

La interpretación es que la insolación modula el calentamiento diurno, pero su efecto sobre la diferencia urbano–rural no es tan directo como el de la temperatura nocturna: ambos entornos reciben radiación solar, mientras que la UHI depende sobre todo de cómo se almacena y libera ese calor.

Presión atmosférica (presMax/presMin)

La presión solo se dispone en algunas estaciones. En Fabra se observa una correlación negativa moderada entre UHI y presMax: los episodios de alta presión (atmósfera más estable) tienden a corresponder con mayor UHI.

Esto tiene sentido físico: las situaciones anticiclónicas favorecen cielos despejados y débiles vientos, condiciones en las que la ciudad se recalienta de día y se ventila poco de noche.

En el aeropuerto y Sabadell la relación es más débil y, debido al menor número de años con datos, la significancia estadística es limitada.

🎯 Conclusión global para el TFG

En conjunto, los resultados confirman que la UHI de Barcelona está controlada principalmente por la dinámica térmica nocturna: las temperaturas mínimas y, sobre todo, la reducción de la amplitud térmica diaria (ATD) son los mejores indicadores de su intensidad.
Otras variables atmosféricas (humedad, insolación y presión) juegan un papel modulador, pero su influencia es secundaria y más difícil de aislar estadísticamente.
Las diferencias entre estaciones (urbana densa, urbana elevada, periurbana costera y valle periurbano) reflejan la interacción entre el clima regional mediterráneo y la morfología urbana y topográfica, reforzando la idea de que la UHI es un fenómeno simultáneamente climático y geográfico.